In [2]:
# MINE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- [sbert](https://www.sbert.net/)
- [training_overview](https://www.sbert.net/docs/sentence_transformer/training_overview.html)
- [loss_overview](https://www.sbert.net/docs/sentence_transformer/loss_overview.html)
- [training_examples](https://www.sbert.net/docs/sentence_transformer/training/examples.html)
- [matryoshka](https://www.sbert.net/examples/sentence_transformer/training/matryoshka/README.html)
- [adaptive_layer](https://www.sbert.net/examples/sentence_transformer/training/adaptive_layer/README.html)
- [training_with_prompts](https://www.sbert.net/examples/sentence_transformer/training/prompts/README.html)
- [training_with_peft](https://www.sbert.net/examples/sentence_transformer/training/peft/README.html)
- [link text](https://)

In [3]:
# @title **All needed imports**
from peft import LoraConfig, TaskType
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss, \
AdaptiveLayerLoss, MatryoshkaLoss, CachedMultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator,\
 NanoBEIREvaluator

## **Embedding-Fine-Tuning-With-PEFT**

### **Load the Model**

In [4]:
# This is optional
model_card=SentenceTransformerModelCardData(
    language='en', license="mit",
    model_name="bge-large-en-v1.5"
)

# A must
model = SentenceTransformer(
    "BAAI/bge-large-en-v1.5",
    model_card_data=model_card
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [5]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [6]:
print(model._first_module().auto_model.config.output_hidden_states)
# Should print: True
# This needs to be adjusted when loading the model

False


In [7]:
model._first_module().auto_model.config

BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dtype": "float32",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.56.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [8]:
# Adjust as needed
use_prompts = True
include_prompts_in_pooling = True

In [9]:

# This is optional
model_card=SentenceTransformerModelCardData(
    language='en', license="mit",
    model_name="bge-large-en-v1.5"
)

# A must
model = SentenceTransformer(
    "BAAI/bge-large-en-v1.5",
    model_card_data=model_card,
    # This fails
    # output_hidden_states=True
)

# This works
# # 2️⃣ Enable hidden states on the first module’s auto_model
# model._first_module().auto_model.config.output_hidden_states = True
model.set_pooling_include_prompt(include_prompts_in_pooling)

In [10]:
print(model._first_module().auto_model.config.output_hidden_states)
print(model._first_module().auto_model.config)
# Should print: True

False
BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dtype": "float32",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.56.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [11]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

### **PEFT Configuration**

In [12]:
peft_config = LoraConfig(
    task_type=TaskType.FEATURE_EXTRACTION,
    inference_mode=False,
    r=64,
    lora_alpha=128,
    lora_dropout=0.1,
)
model.add_adapter(peft_config)

In [16]:
print(model._first_module().auto_model.config)

BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dtype": "float32",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.56.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [17]:
# @title **Load your dataset**
dataset = load_dataset("sentence-transformers/natural-questions", split="train")
dataset_dict = dataset.train_test_split(test_size=0.2, seed=30)

In [18]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['query', 'answer'],
        num_rows: 80184
    })
    test: Dataset({
        features: ['query', 'answer'],
        num_rows: 20047
    })
})

In [19]:
train_dataset = dataset_dict['train']
train_dataset

Dataset({
    features: ['query', 'answer'],
    num_rows: 80184
})

In [20]:
validation_dataset = dataset_dict['test']
validation_dataset

Dataset({
    features: ['query', 'answer'],
    num_rows: 20047
})

### **Data Inspection**

In [21]:
# train
dataset_dict.column_names

{'train': ['query', 'answer'], 'test': ['query', 'answer']}

In [22]:
def random_examples_selector(given_dataset):
   from random import randint
   query = given_dataset['query'][randint(0, len(given_dataset))]
   answer = given_dataset['answer'][randint(0, len(given_dataset))]

   print(f"query: {query}")
   print(f"answer: {answer}")

In [23]:
random_examples_selector(train_dataset)

query: what is the rhythm of the opening motive of beethovens fifth symphony
answer: Michael Emerson Michael Emerson (born September 7, 1954)[3] is an American film and television actor who is best known for his roles as serial killer William Hinks on The Practice, Benjamin Linus on Lost, Zep Hindle in the first Saw film, and Harold Finch on the CBS series Person of Interest. Emerson has also worked extensively in theater and narration. He has won two Primetime Emmy Awards and been nominated for three others, as well as receiving other awards and nominations.


In [24]:
random_examples_selector(validation_dataset)

query: where did the navy seals get their name
answer: Circular motion In uniform circular motion, total acceleration of an object in a circular path is equal to the radial acceleration. Due to the presence of tangential acceleration in non uniform circular motion, that does not hold true any more. To find the total acceleration of an object in non uniform circular, find the vector sum of the tangential acceleration and the radial acceleration.


In [25]:
def examples_selector(given_dataset,index:int):
   query = given_dataset['query'][index]
   answer = given_dataset['answer'][index]

   print(f"query: {query}")
   print(f"answer: {answer}")

In [26]:
examples_selector(train_dataset,1)

query: what was the purpose of the three-field system
answer: Three-field system The three-field system is a regime of crop rotation that was used in medieval and early-modern Europe. Crop rotation is the practice of growing a series of different types of crops in the same area in sequential seasons. Under this system, the arable land of an estate or village was divided into three large fields: one was planted in the autumn with winter wheat or rye; the second field was planted with other crops such as peas, lentils, or beans; and the third was left fallow, in order to allow the soil of that field to regain its nutrients. With each rotation, the field would be used differently, so that a field would be planted for two out of the three years used, whilst one year it "rested". Previously a "two field system" had been in place, with half the land being left fallow. The three field system allowed farmers to plant more crops and therefore to increase production and legumes have the ability 

In [27]:
examples_selector(validation_dataset,1)

query: who is doing the half time super bowl
answer: Super Bowl LII halftime show The Super Bowl LII Halftime Show (officially known as the Pepsi Super Bowl LII Halftime Show) took place on February 4, 2018 at U.S. Bank Stadium in Minneapolis, Minnesota, as part of Super Bowl LII. Justin Timberlake was the featured performer, as confirmed by the National Football League (NFL) on October 22, 2017.[1] It was televised nationally by NBC.


### **Loss Definition**

In [28]:
base_loss = CachedMultipleNegativesRankingLoss(model, mini_batch_size=16)

# Could include a Matryoshka Approach Loss/ Adaptive Loss or a combo of them
# matryoshka_dims = [1024,768, 512, 256, 128, 64]
# loss =  MatryoshkaLoss(model=model,
#                          loss=base_loss,
#                          matryoshka_dims=matryoshka_dims)

In [29]:
base_loss

CachedMultipleNegativesRankingLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 512, 'do_lower_case': True, 'architecture': 'BertModel'})
    (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
    (2): Normalize()
  )
  (cross_entropy_loss): CrossEntropyLoss()
)

### **Setting up Weights and Biases for Logging**

In [30]:
import wandb

In [31]:
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dannyai (dannyai-danny-the-analyst) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [32]:
import os
# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="embedding_fine_tuning_with_peft_bge_large_en_v1.5"

# save your trained model checkpoint to wandb
# os.environ["WANDB_LOG_MODEL"]="true" # throws an error, must use 'checkpoint' or 'end'
os.environ["WANDB_LOG_MODEL"]="checkpoint"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

### **Specify Training Arguments**

[Training Arguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments.save_total_limit)

In [33]:
if use_prompts:
  query_prompt = "query: "
  corpus_prompt = "document: "
  prompts = {
        "query": query_prompt,
        "answer": corpus_prompt,
  }

In [34]:
training_args = SentenceTransformerTrainingArguments(
    # num_train_epochs=1, # full training,
    prompts = prompts if use_prompts else None,
    max_steps=100, # reduced from 300 to 100
    per_device_train_batch_size=5, # reduced from 16 to 5
    per_device_eval_batch_size=5, # reduced from 16 to 5
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    # Some optional tracking and debugging parameters:
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=3,
    logging_strategy="steps",
    logging_steps=100,
    # logging_first_step=True,
    load_best_model_at_end=True,
    seed = 30,
    output_dir = "outputs",
    run_name="embedding_fine_tuning_with_peft_bge_large_en_v1.5",
    report_to=["wandb"] # reporting to Weights and biases project
)

In [36]:
print(training_args)

SentenceTransformerTrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
batch_sampler=BatchSamplers.NO_DUPLICATES,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=False,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=100,
eval_

### **Evaluator**

- [TripletEvaluator](https://www.sbert.net/docs/package_reference/sentence_transformer/evaluation.html#sentence_transformers.evaluation.TripletEvaluator)
- [SimilarityFunction](https://www.sbert.net/docs/package_reference/sparse_encoder/SparseEncoder.html#sentence_transformers.SimilarityFunction)
- [NanoBEIREvaluator](https://www.sbert.net/docs/package_reference/sentence_transformer/evaluation.html#sentence_transformers.evaluation.NanoBEIREvaluator)
- [Example Code](https://github.com/UKPLab/sentence-transformers/blob/master/examples/sentence_transformer/training/peft/training_gooaq_lora.py)

In [37]:
from sentence_transformers.evaluation import SimilarityFunction
# Initialise the evaluator
 # dataset evaluator is from hugggingface
 # https://huggingface.co/collections/zeta-alpha-ai/nanobeir-66e1a0af21dfd93e620cd9f6
 # Will use only one dataset "QuoraRetrieval"
datasets  = ['QuoraRetrieval']
val_evaluator = NanoBEIREvaluator(
    dataset_names = datasets,
    query_prompts=query_prompt if use_prompts else None,
    corpus_prompts=corpus_prompt if use_prompts else None,
    batch_size=50,
    show_progress_bar=True
)

Loading NanoBEIR datasets:   0%|          | 0/1 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

corpus/train-00000-of-00001.parquet:   0%|          | 0.00/229k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5046 [00:00<?, ? examples/s]

queries/train-00000-of-00001.parquet:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

qrels/train-00000-of-00001.parquet:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/70 [00:00<?, ? examples/s]

In [38]:
# @title **Create an evaluator & evaluate the base model**
val_evaluator(model)

Evaluating datasets:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Corpus Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/101 [00:00<?, ?it/s]


Evaluating datasets: 100%|██████████| 1/1 [00:21<00:00, 21.32s/it]


{'NanoQuoraRetrieval_cosine_accuracy@1': 0.92,
 'NanoQuoraRetrieval_cosine_accuracy@3': 0.98,
 'NanoQuoraRetrieval_cosine_accuracy@5': 0.98,
 'NanoQuoraRetrieval_cosine_accuracy@10': 1.0,
 'NanoQuoraRetrieval_cosine_precision@1': 0.92,
 'NanoQuoraRetrieval_cosine_precision@3': 0.4133333333333333,
 'NanoQuoraRetrieval_cosine_precision@5': 0.25199999999999995,
 'NanoQuoraRetrieval_cosine_precision@10': 0.13999999999999999,
 'NanoQuoraRetrieval_cosine_recall@1': 0.8073333333333332,
 'NanoQuoraRetrieval_cosine_recall@3': 0.9520000000000001,
 'NanoQuoraRetrieval_cosine_recall@5': 0.9553333333333334,
 'NanoQuoraRetrieval_cosine_recall@10': 1.0,
 'NanoQuoraRetrieval_cosine_ndcg@10': 0.9582650610304665,
 'NanoQuoraRetrieval_cosine_mrr@10': 0.9520000000000001,
 'NanoQuoraRetrieval_cosine_map@100': 0.9383253968253967,
 'NanoBEIR_mean_cosine_accuracy@1': 0.92,
 'NanoBEIR_mean_cosine_accuracy@3': 0.98,
 'NanoBEIR_mean_cosine_accuracy@5': 0.98,
 'NanoBEIR_mean_cosine_accuracy@10': 1.0,
 'NanoBEIR_m

In [39]:
# @title **Create a trainer & train**

trainer = SentenceTransformerTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    evaluator=val_evaluator,
    loss=base_loss
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [40]:
trainer.train()

Step,Training Loss,Validation Loss,Nanoquoraretrieval Cosine Accuracy@1,Nanoquoraretrieval Cosine Accuracy@3,Nanoquoraretrieval Cosine Accuracy@5,Nanoquoraretrieval Cosine Accuracy@10,Nanoquoraretrieval Cosine Precision@1,Nanoquoraretrieval Cosine Precision@3,Nanoquoraretrieval Cosine Precision@5,Nanoquoraretrieval Cosine Precision@10,Nanoquoraretrieval Cosine Recall@1,Nanoquoraretrieval Cosine Recall@3,Nanoquoraretrieval Cosine Recall@5,Nanoquoraretrieval Cosine Recall@10,Nanoquoraretrieval Cosine Ndcg@10,Nanoquoraretrieval Cosine Mrr@10,Nanoquoraretrieval Cosine Map@100,Nanobeir Mean Cosine Accuracy@1,Nanobeir Mean Cosine Accuracy@3,Nanobeir Mean Cosine Accuracy@5,Nanobeir Mean Cosine Accuracy@10,Nanobeir Mean Cosine Precision@1,Nanobeir Mean Cosine Precision@3,Nanobeir Mean Cosine Precision@5,Nanobeir Mean Cosine Precision@10,Nanobeir Mean Cosine Recall@1,Nanobeir Mean Cosine Recall@3,Nanobeir Mean Cosine Recall@5,Nanobeir Mean Cosine Recall@10,Nanobeir Mean Cosine Ndcg@10,Nanobeir Mean Cosine Mrr@10,Nanobeir Mean Cosine Map@100
100,0.015600,0.006718,0.880000,0.980000,0.980000,1.000000,0.880000,0.413333,0.252000,0.140000,0.767333,0.952000,0.955333,1.000000,0.943561,0.929524,0.919405,0.880000,0.980000,0.980000,1.000000,0.880000,0.413333,0.252000,0.140000,0.767333,0.952000,0.955333,1.000000,0.943561,0.929524,0.919405


Evaluating datasets:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Corpus Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/101 [00:00<?, ?it/s]


Evaluating datasets: 100%|██████████| 1/1 [00:30<00:00, 30.84s/it]
wandb: Adding directory to artifact (outputs/checkpoint-100)... Done. 0.3s


TrainOutput(global_step=100, training_loss=0.015634404420852663, metrics={'train_runtime': 1760.9983, 'train_samples_per_second': 0.284, 'train_steps_per_second': 0.057, 'total_flos': 0.0, 'train_loss': 0.015634404420852663, 'epoch': 0.00623558022073954})

In [41]:
# @title **Evaluate the trained model on the test set**
test_evaluator = NanoBEIREvaluator(
    dataset_names = datasets,
    query_prompts=query_prompt if use_prompts else None,
    corpus_prompts=corpus_prompt if use_prompts else None,
    batch_size=50,
    show_progress_bar=True
)

In [42]:
test_evaluator(model)

Evaluating datasets:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Corpus Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/101 [00:00<?, ?it/s]


Evaluating datasets: 100%|██████████| 1/1 [00:29<00:00, 29.86s/it]


{'NanoQuoraRetrieval_cosine_accuracy@1': 0.88,
 'NanoQuoraRetrieval_cosine_accuracy@3': 0.98,
 'NanoQuoraRetrieval_cosine_accuracy@5': 0.98,
 'NanoQuoraRetrieval_cosine_accuracy@10': 1.0,
 'NanoQuoraRetrieval_cosine_precision@1': 0.88,
 'NanoQuoraRetrieval_cosine_precision@3': 0.4133333333333333,
 'NanoQuoraRetrieval_cosine_precision@5': 0.25199999999999995,
 'NanoQuoraRetrieval_cosine_precision@10': 0.13999999999999999,
 'NanoQuoraRetrieval_cosine_recall@1': 0.7673333333333332,
 'NanoQuoraRetrieval_cosine_recall@3': 0.9520000000000001,
 'NanoQuoraRetrieval_cosine_recall@5': 0.9553333333333334,
 'NanoQuoraRetrieval_cosine_recall@10': 1.0,
 'NanoQuoraRetrieval_cosine_ndcg@10': 0.9435612217207588,
 'NanoQuoraRetrieval_cosine_mrr@10': 0.9295238095238095,
 'NanoQuoraRetrieval_cosine_map@100': 0.919404761904762,
 'NanoBEIR_mean_cosine_accuracy@1': 0.88,
 'NanoBEIR_mean_cosine_accuracy@3': 0.98,
 'NanoBEIR_mean_cosine_accuracy@5': 0.98,
 'NanoBEIR_mean_cosine_accuracy@10': 1.0,
 'NanoBEIR_me

### **Model Saving**

In [43]:
# @title **Save the trained model**
model.save_pretrained("embedding_fine_tuning_with_peft_bge_large_en_v1.5")

In [46]:
from google.colab import userdata
from huggingface_hub import login
hf_token = userdata.get('HF_TOKEN')
if hf_token:
   login(hf_token)
   print("Successfully logged in to Hugging Face!")
else:
   print("Token is not set. Please save the token first.")

Successfully logged in to Hugging Face!


In [47]:
# @title **Push it to the Hugging Face Hub**
# do not create repo on hugging face
model.push_to_hub("DannyAI/embedding_fine_tuning_with_peft_bge_large_en_v1.5", exist_ok=True)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   2%|2         |  554kB / 25.2MB            

'https://huggingface.co/DannyAI/embedding_fine_tuning_with_peft_bge_large_en_v1.5/commit/d20edb4325ab9d910196edd81a5b85e9ed32def6'

In [53]:
# @title **Load PEFT model directly**
model = SentenceTransformer("DannyAI/embedding_fine_tuning_with_peft_bge_large_en_v1.5")
embeddings = model.encode(["Who is the greatest naruto fan?", "Daniel is the greatest naruto fan"])
embeddings.shape

(2, 1024)

In [54]:
embeddings

array([[ 0.00650023,  0.01436641, -0.00454454, ..., -0.04594097,
        -0.0139524 , -0.02504645],
       [ 0.00265199, -0.01463662,  0.01979103, ..., -0.04177105,
        -0.01575905, -0.00444059]], dtype=float32)

In [55]:
# @title **Load PEFT model with base model**
model = SentenceTransformer("BAAI/bge-large-en-v1.5")
model.load_adapter("DannyAI/embedding_fine_tuning_with_peft_bge_large_en_v1.5")
embeddings = model.encode(["Who is the greatest naruto fan?", "Daniel is the greatest naruto fan"])
embeddings.shape

(2, 1024)

In [56]:
embeddings

array([[ 0.00650023,  0.01436641, -0.00454454, ..., -0.04594097,
        -0.0139524 , -0.02504645],
       [ 0.00265199, -0.01463662,  0.01979103, ..., -0.04177105,
        -0.01575905, -0.00444059]], dtype=float32)

In most cases, the former is easiest, as it will work regardless of whether the model is an adapter model or not.